#BERT development praparation
BERT needs python3.6, tensorflow1.13.1, pytorch1.10.0, pytorch-transformers1.2.0 to run properly.

In [2]:
!python -V
!sudo apt-get install python3.6
!sudo update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.6 3
#!sudo update-alternatives --config python3
!sudo apt-get install python3.6-distutils
!sudo apt install python3-pip
!python -m pip install --upgrade pip
!pip uninstall six -y
#!pip install tensorflow==1.13.1
!pip install tensorflow-gpu==1.13.1
!python -V

Python 3.10.11
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libpython3.6-minimal libpython3.6-stdlib python3.6-minimal
Suggested packages:
  python3.6-venv binfmt-support
The following NEW packages will be installed:
  libpython3.6-minimal libpython3.6-stdlib python3.6 python3.6-minimal
0 upgraded, 4 newly installed, 0 to remove and 24 not upgraded.
Need to get 4,294 kB of archives.
After this operation, 22.1 MB of additional disk space will be used.
Get:1 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.6-minimal amd64 3.6.15-1+focal3 [569 kB]
Get:2 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 python3.6-minimal amd64 3.6.15-1+focal3 [1,718 kB]
Get:3 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 libpython3.6-stdlib amd64 3.6.15-1+focal3 [1,758 kB]
Get:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal/main amd64 pyt

Python 3.6.15


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Install Cuda10.0 and CuDNN7.4 to cope with Tensorflow1.13.1.

In [ ]:
%cd /content/drive/MyDrive/libs
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0
#%cd /content/drive/MyDrive/Code
#!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/libcudnn7-dev_7.4.2.24-1+cuda10.0_amd64.deb
#!wget https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/libcudnn7_7.4.2.24-1+cuda10.0_amd64.deb
!dpkg -i libcudnn7_7.4.2.24-1+cuda10.0_amd64.deb
!dpkg -i libcudnn7-dev_7.4.2.24-1+cuda10.0_amd64.deb

0.GLUE data

In [ ]:
# %cd /content/drive/MyDrive/Code
# %pwd
# for i in ["CoLA", "SST", "MRPC", "QQP", "STS", "MNLI", "QNLI", "RTE", "WNLI", "diagnostic"]:
#   !python download_glue.py --data_dir data --tasks $i

/content/drive/MyDrive/Code
	Completed!
	Completed!
Processing MRPC...
	Completed!
	Completed!
	Completed!
	Note (12/10/20): This script no longer downloads SNLI. You will need to manually download and format the data to use SNLI.
	Completed!
	Completed!
	Completed!
	Completed!
	Completed!


In [ ]:
%cd /content/drive/MyDrive/Code_pretrain
!wget https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-4_H-256_A-4.zip
!unzip uncased_L-4_H-256_A-4.zip -d uncased_L-4_H-256_A-4

/content/drive/MyDrive/Code_pretrain
--2023-05-22 06:21:10--  https://storage.googleapis.com/bert_models/2020_02_20/uncased_L-4_H-256_A-4.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.194.128, 74.125.200.128, 74.125.68.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.194.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41854126 (40M) [application/zip]
Saving to: ‘uncased_L-4_H-256_A-4.zip’

uncased_L-4_H-256_A 100%[===================>]  39.92M  12.5MB/s    in 3.2s    

2023-05-22 06:21:14 (12.5 MB/s) - ‘uncased_L-4_H-256_A-4.zip’ saved [41854126/41854126]

Archive:  uncased_L-4_H-256_A-4.zip
  inflating: uncased_L-4_H-256_A-4/bert_model.ckpt.data-00000-of-00001  
  inflating: uncased_L-4_H-256_A-4/bert_config.json  
  inflating: uncased_L-4_H-256_A-4/vocab.txt  
  inflating: uncased_L-4_H-256_A-4/bert_model.ckpt.index  


#BERT-Mini
Use a mini model to walk through finetuning steps.

In [ ]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=CoLA \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/CoLA \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:***** Eval results *****
# INFO:tensorflow:  eval_accuracy = 0.6912752
# INFO:tensorflow:  eval_loss = 0.6332376
# INFO:tensorflow:  global_step = 2137
# INFO:tensorflow:  loss = 0.63339067

# INFO:tensorflow:  eval_accuracy = 0.6912752
# INFO:tensorflow:  eval_loss = 0.63406587
# INFO:tensorflow:  global_step = 2137
# INFO:tensorflow:  loss = 0.63434684

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [ ]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=SST-2 \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/SST-2 \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.7912844
# INFO:tensorflow:  eval_loss = 0.6541138
# INFO:tensorflow:  global_step = 16837
# INFO:tensorflow:  loss = 0.6541138

# INFO:tensorflow:  eval_accuracy = 0.83600914
# INFO:tensorflow:  eval_loss = 0.6130721
# INFO:tensorflow:  global_step = 16837
# INFO:tensorflow:  loss = 0.6130721


/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [ ]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=MRPC \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/MRPC \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.70343137
# INFO:tensorflow:  eval_loss = 0.605878
# INFO:tensorflow:  global_step = 917
# INFO:tensorflow:  loss = 0.605878

# INFO:tensorflow:  eval_accuracy = 0.72794116
# INFO:tensorflow:  eval_loss = 0.5443143
# INFO:tensorflow:  global_step = 917
# INFO:tensorflow:  loss = 0.5443143

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

#MNLI is slow

In [16]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=MNLI \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/MNLI \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.64605194
# INFO:tensorflow:  eval_loss = 0.80982023
# INFO:tensorflow:  global_step = 98175
# INFO:tensorflow:  loss = 0.8097828

# INFO:tensorflow:  eval_accuracy = 0.71421295
# INFO:tensorflow:  eval_loss = 0.7193103
# INFO:tensorflow:  global_step = 98175
# INFO:tensorflow:  loss = 0.7192658

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [5]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=MNLI-MM \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/MNLI \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.6647681
# INFO:tensorflow:  eval_loss = 0.7852047
# INFO:tensorflow:  global_step = 98175
# INFO:tensorflow:  loss = 0.7852047

# INFO:tensorflow:  eval_accuracy = 0.5509561
# INFO:tensorflow:  eval_loss = 0.94312745
# INFO:tensorflow:  global_step = 98175
# INFO:tensorflow:  loss = 0.94312745

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [18]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=STS-B \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/STS-B \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.42733333
# INFO:tensorflow:  eval_loss = 1.4847277
# INFO:tensorflow:  global_step = 1437
# INFO:tensorflow:  loss = 1.485607

# INFO:tensorflow:  eval_accuracy = 0.45266667
# INFO:tensorflow:  eval_loss = 1.2799171
# INFO:tensorflow:  global_step = 1437
# INFO:tensorflow:  loss = 1.2798506

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [ ]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=QQP \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/QQP \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.79809546
# INFO:tensorflow:  eval_loss = 0.48185706
# INFO:tensorflow:  global_step = 90961
# INFO:tensorflow:  loss = 0.48184583

# INFO:tensorflow:  eval_accuracy = 0.76094484
# INFO:tensorflow:  eval_loss = 0.5269344
# INFO:tensorflow:  global_step = 90961
# INFO:tensorflow:  loss = 0.52696335

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-23640fa6337e>", line 1, in <cell line: 1>
    get_ipython().run_line_magic('cd', '/content/drive/MyDrive/Code_pretrain')
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2418, in run_line_magic
    result = fn(*args, **kwargs)
  File "<decorator-gen-85>", line 2, in cd
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magic.py", line 187, in <lambda>
    call = lambda f, *a, **k: f(*a, **k)
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py", line 342, in cd
    oldcwd = os.getcwd()
OSError: [Errno 107] Transport endpoint is not connected

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/

In [19]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=QNLI \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/QNLI \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.77631336
# INFO:tensorflow:  eval_loss = 0.54709363
# INFO:tensorflow:  global_step = 26185
# INFO:tensorflow:  loss = 0.5471753

# INFO:tensorflow:  eval_accuracy = 0.8321435
# INFO:tensorflow:  eval_loss = 0.4846125
# INFO:tensorflow:  global_step = 26185
# INFO:tensorflow:  loss = 0.48469925

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [3]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=WNLI \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/WNLI \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.4225352
# INFO:tensorflow:  eval_loss = 0.70586663
# INFO:tensorflow:  global_step = 158
# INFO:tensorflow:  loss = 0.70596206

# INFO:tensorflow:  eval_accuracy = 0.4084507
# INFO:tensorflow:  eval_loss = 0.7177324
# INFO:tensorflow:  global_step = 158
# INFO:tensorflow:  loss = 0.7176453

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python

In [21]:
%cd /content/drive/MyDrive/Code_pretrain
%pwd
%env BERT_BASE_DIR=uncased_L-4_H-256_A-4
%env GLUE_DIR=data
!rm -rf /content/trained_output

!python3 run_classifier.py \
  --task_name=RTE \
  --do_train=true \
  --do_eval=true \
  --data_dir=$GLUE_DIR/RTE \
  --vocab_file=$BERT_BASE_DIR/vocab.txt \
  --bert_config_file=$BERT_BASE_DIR/bert_config.json \
  --init_checkpoint=$BERT_BASE_DIR/bert_model.ckpt \
  --max_seq_length=128 \
  --train_batch_size=4 \
  --learning_rate=2e-5 \
  --num_train_epochs=1.0 \
  --output_dir=/content/trained_output

# result metric.
# INFO:tensorflow:  eval_accuracy = 0.59205776
# INFO:tensorflow:  eval_loss = 0.6777811
# INFO:tensorflow:  global_step = 622
# INFO:tensorflow:  loss = 0.6773494

# INFO:tensorflow:  eval_accuracy = 0.6101083
# INFO:tensorflow:  eval_loss = 0.6598809
# INFO:tensorflow:  global_step = 622
# INFO:tensorflow:  loss = 0.65960556

/content/drive/MyDrive/Code_pretrain
env: BERT_BASE_DIR=uncased_L-4_H-256_A-4
env: GLUE_DIR=data
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python